신문사하나정해서

10페이지 컨텐츠를 수집해서 신문내용긁어오기

## #01. 준비과정
### [1] 패키지 참조


In [3]:
import datetime as dt
import requests
import os
import time
from bs4 import BeautifulSoup
import markdownify as mf
from tqdm.notebook import tqdm
import re
import concurrent.futures as futures

### [2] 접속할 데이터의 URL

In [4]:

url_page = "https://it.chosun.com/news/articleList.html?page={pageNumber}&total=30235&box_idxno=&sc_section_code=S1N4&view_type=sm" # 페이지


## #02. 데이터 요청하기
### [1] 세션 생성

In [5]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

In [6]:
def getUrl(session, url_page, url_set):
    domain = "https://it.chosun.com"
    
    try:
        r = session.get(url_page)
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)

    body = soup.select("#section-list")[0]
    title = body.text.strip()
    title = title.split("\n")
    for i,v in enumerate(title):
        title[i] = v.strip()
    new_title = []
    for i in title:
        if i: new_title.append(i)
    title_list = []
    datetime_list = []

    for i in range(len(new_title)//4):
        title_list.append(new_title[i*4])
        datetime = new_title[i*4+3].replace('.','').replace(':','').replace(' ','')
        datetime_list.append(datetime)

    urlset = soup.select("#section-list > .type > .item > a[href]")
    articleUrl = []
    for v in urlset:
        articleUrl.append(domain + v.attrs['href'])

    mydata = []

    title_loss = '\\/:*?\"<>|' 
    
    for i in range(20):
        
        A = title_list[i]
        for j in title_loss: # 파일명에 붙일 수 없는 특수문자 제거
            A = A.replace(j,'')

        # A = re.sub(r"[^ㄱ-ㅎ가-힣0-9a-zA-Z\s]", "", title_list[i]) # 정규표현식을 이용하여 한글/숫자/영어/띄어쓰기만 남긴다. => 특수문자 모두 제거됨
        # re.sub(정규표현식, 변경문자열, 원본문자열) : 원본문자열에서 정규표현식에 해당되는것을 변경문자(열)로 치환함
        # A = re.sub('[\\/:*?<>\"\']','',title_list[i])
        if articleUrl[i] in url_set: continue # 중복된 url 제거
        url_set.add(articleUrl[i])
        mydict = {
            "title" : A,
            "datetime" : datetime_list[i],
            "url" : articleUrl[i]
        }
        mydata.append(mydict)

    time.sleep(0.05)
    return mydata
    
 



In [7]:
def getArticleBody(session, url):
    domain = "https://it.chosun.com"
    
    try:
        r = session.get(url)
        
        if r.status_code != 200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)
    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)

    # 게시글 영역 추출
    r.encoding = "utf-8"
    soup = BeautifulSoup(r.text)
    content = soup.select(".article-body")[0]

    # 문자열로 변환
    body = str(content)
    # 이미지부분 조정

    rebody = body.replace('src="/',f'src="{domain}/')
    # 마크다운으로 변경

    time.sleep(0.05)  # 오래돌아야하는 경우 컴퓨터의 다른 프로그램을 위한 메모리를 남겨주기 위해 타임슬립을 걸어야한다.
    return mf.markdownify(rebody)

기존에 하던 식으로 하면 모든 데이터를 메모리에 저장하기때문에 효율적이지 못하다

따라서, 따로 메모리에 저장하지 않고 바로 파일을 저장한다.

In [8]:
dirname = dt.datetime.now().strftime('신문기사_수집_%y%m%d_%H%M%S')

if not os.path.exists(dirname):
    os.mkdir(dirname)

articleUrlSet = []
with futures.ThreadPoolExecutor(max_workers=10) as executor:

    for i in range(1,11):

        pageNumber = i
        url_set = set()
        targetUrl = url_page.format(pageNumber = i)
        for v in getUrl(session, targetUrl,url_set):
            fu = executor.submit(getArticleBody, session, v['url'])
            md = fu.result()
            
            fname = f"{dirname}/{v['datetime']}_{v['title']}.md"
            fname = fname.replace('-','').replace(' ','_')
            
            with open(fname, 'w', encoding='utf-8') as f:
                f.write(md)
            time.sleep(0.05)

